# 예보 데이터 불러오기
[한국동서발전 태양광 발전량 현황 인증키 확인](https://data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15003553#tab-layer-openapi)

[인증키 주소(로그인필요..)](https://data.go.kr/iim/api/selectAPIAcountView.do)

In [140]:
! pip install xmltodict

In [141]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict

In [1]:
# API
ServiceKey = input('encoded api key를 입력해주세요 > ')

encoded api key를 입력해주세요 > a


# 1번째 url 써보기

* 각 발전소별로 2018-01-01 ~ 2019-08-31 까지 데이터가 있음

In [143]:
def get_energy_json(page, perpage):
    # JSON으로 파일받은 뒤 정리하기
    url_base1 = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
    url_base2 = 'https://api.odcloud.kr/api/15003553/v1/uddi:b6eb07ba-4892-4d15-aa24-b8398a5ceffc'
    url_base3 = 'https://api.odcloud.kr/api/15003553/v1/uddi:ad544f1e-b5cc-4f74-acac-a66aca3e8b40'
    url_base = url_base1
    
    key = ServiceKey
    queryParams = urlencode({
        quote_plus('page'): page,      # 불러오고 싶은만큼 가져오기
        quote_plus('perPage'): perpage,
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')

    data = pd.read_json(results)
    
    return data

def get_energy_data(data_json):
    data_energy = pd.DataFrame()
    for i in range(len(data_json)):
        # print(f'{i}일차 데이터 가져오기 시작!')
        data = data_json.copy()
        
        dict_data = data.loc[i, 'data']
        time = dict_data.pop('시간')
        region = dict_data.pop('태양광명')

        time_data = pd.DataFrame([dict_data]).T
        time_data.reset_index(inplace=True)
        time_data.columns = ['hours', '발전량']

        time_data['시간'] = time
        time_data['date'] = time_data['시간'] + ' ' + time_data['hours'] + ':00:00'
        time_data['hours'] = time_data['hours'].astype('int64')
        time_data['태양광명'] = region
        time_data.sort_values('hours')

        col_return = ['date', '발전량', '태양광명']

    #     print(time_data[col_return])
        data_energy = pd.concat([data_energy, time_data[col_return]])
    
    return data_energy


In [144]:
y_all = pd.DataFrame()
for i in range(10):
    x = get_energy_json(i+1, 1000)
    y = get_energy_data(x)
    y_all = pd.concat([y_all, y])
print(y_all.shape)

(196848, 3)


In [145]:
names = y_all['태양광명'].value_counts().index.to_list()
for name in names:
    print(y_all[y_all['태양광명'] == name].iloc[:2, :])
    print(y_all[y_all['태양광명'] == name].iloc[-2:, :])
    print('\n')


                  date  발전량    태양광명
0   2018-01-01 1:00:00    0  광양항태양광
1  2018-01-01 10:00:00  427  광양항태양광
                  date  발전량    태양광명
22  2019-08-31 8:00:00  202  광양항태양광
23  2019-08-31 9:00:00  383  광양항태양광


                  date  발전량        태양광명
0   2018-01-01 1:00:00    0  수원환경사업소태양광
1  2018-01-01 10:00:00  552  수원환경사업소태양광
                  date  발전량        태양광명
22  2019-08-31 8:00:00  227  수원환경사업소태양광
23  2019-08-31 9:00:00  511  수원환경사업소태양광


                  date  발전량       태양광명
0   2018-01-01 1:00:00    0  황금물류센터태양광
1  2018-01-01 10:00:00  187  황금물류센터태양광
                  date  발전량       태양광명
22  2019-08-31 8:00:00  126  황금물류센터태양광
23  2019-08-31 9:00:00  217  황금물류센터태양광


                  date  발전량     태양광명
0   2018-01-01 1:00:00    0  당진수상태양광
1  2018-01-01 10:00:00  180  당진수상태양광
                  date  발전량     태양광명
22  2019-08-31 8:00:00   37  당진수상태양광
23  2019-08-31 9:00:00  162  당진수상태양광


                  date  발전량          태양광명
0   2018-01-01 1:00:00    0  동해하수종말처리장

# 2번째 url 써보기

* 각 발전소별로 2015-01-01 ~ 2017-12-31 까지 데이터가 있음

In [146]:
def get_energy_json(page, perpage):
    # JSON으로 파일받은 뒤 정리하기
    url_base1 = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
    url_base2 = 'https://api.odcloud.kr/api/15003553/v1/uddi:b6eb07ba-4892-4d15-aa24-b8398a5ceffc'
    url_base3 = 'https://api.odcloud.kr/api/15003553/v1/uddi:ad544f1e-b5cc-4f74-acac-a66aca3e8b40'
    url_base = url_base2
    
    key = ServiceKey
    queryParams = urlencode({
        quote_plus('page'): page,      # 불러오고 싶은만큼 가져오기
        quote_plus('perPage'): perpage,
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')

    data = pd.read_json(results)
    
    return data

def get_energy_data(data_json):
    data_energy = pd.DataFrame()
    for i in range(len(data_json)):
        # print(f'{i}일차 데이터 가져오기 시작!')
        data = data_json.copy()
        
        dict_data = data.loc[i, 'data']
        time = dict_data.pop('시간')
        region = dict_data.pop('태양광명')
        elec = dict_data.pop('용량(MW)')

        time_data = pd.DataFrame([dict_data]).T
        time_data.reset_index(inplace=True)
        time_data.columns = ['hours', '발전량']

        time_data['시간'] = time
        time_data['date'] = time_data['시간'] + ' ' + time_data['hours'] + ':00:00'
        time_data['hours'] = time_data['hours'].astype('int64')
        time_data['태양광명'] = region
        time_data['용량(MW)'] = elec
        time_data.sort_values('hours')

        col_return = ['date', '발전량', '태양광명', '용량(MW)']

    #     print(time_data[col_return])
        data_energy = pd.concat([data_energy, time_data[col_return]])
    
    return data_energy


In [147]:
y_all = pd.DataFrame()
for i in range(10):
    x = get_energy_json(i+1, 1000)
    y = get_energy_data(x)
    y_all = pd.concat([y_all, y])
print(y_all.shape)

(234912, 4)


In [148]:
names = y_all['태양광명'].value_counts().index.to_list()
for name in names:
    print(y_all[y_all['태양광명'] == name].iloc[:2, :])
    print(y_all[y_all['태양광명'] == name].iloc[-2:, :])
    print('\n')


                  date  발전량   태양광명 용량(MW)
0   2015-01-01 1:00:00    0  동해태양광      1
1  2015-01-01 10:00:00  170  동해태양광      1
                  date  발전량   태양광명 용량(MW)
22  2017-12-31 8:00:00    0  동해태양광      1
23  2017-12-31 9:00:00   15  동해태양광      1


                  date  발전량    태양광명 용량(MW)
0   2015-01-01 1:00:00    0  광양항태양광    2.3
1  2015-01-01 10:00:00  430  광양항태양광    2.3
                  date  발전량    태양광명 용량(MW)
22  2017-12-31 8:00:00    0  광양항태양광    2.3
23  2017-12-31 9:00:00  112  광양항태양광    2.3


                  date  발전량        태양광명 용량(MW)
0   2015-01-01 1:00:00    0  수원환경사업소태양광    1.5
1  2015-01-01 10:00:00  604  수원환경사업소태양광    1.5
                  date  발전량        태양광명 용량(MW)
22  2017-12-31 8:00:00    0  수원환경사업소태양광    1.5
23  2017-12-31 9:00:00   83  수원환경사업소태양광    1.5


                  date  발전량       태양광명 용량(MW)
0   2015-01-01 1:00:00    0  황금물류센터태양광    1.1
1  2015-01-01 10:00:00  230  황금물류센터태양광    1.1
                  date  발전량       태양광명 용량(MW)
22  2017-12-31 8:0

# 3번째 url 써보기

* 각 발전소별로 2017-07-01 ~ 2019-08-31 까지 데이터가 있음

In [149]:
def get_energy_json(page, perpage):
    # JSON으로 파일받은 뒤 정리하기
    url_base1 = 'https://api.odcloud.kr/api/15003553/v1/uddi:34a871be-f9c5-497a-9c69-a9770ff0ec8d'
    url_base2 = 'https://api.odcloud.kr/api/15003553/v1/uddi:b6eb07ba-4892-4d15-aa24-b8398a5ceffc'
    url_base3 = 'https://api.odcloud.kr/api/15003553/v1/uddi:ad544f1e-b5cc-4f74-acac-a66aca3e8b40'
    url_base = url_base3
    
    key = ServiceKey
    queryParams = urlencode({
        quote_plus('page'): page,      # 불러오고 싶은만큼 가져오기
        quote_plus('perPage'): perpage,
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')

    data = pd.read_json(results)
    
    return data

def get_energy_data(data_json):
    data_energy = pd.DataFrame()
    for i in range(len(data_json)):
        # print(f'{i}일차 데이터 가져오기 시작!')
        data = data_json.copy()
        
        dict_data = data.loc[i, 'data']
        time = dict_data.pop('일시')
        region = dict_data.pop('구분')

        time_data = pd.DataFrame([dict_data]).T
        time_data.reset_index(inplace=True)
        time_data.columns = ['hours', '발전량']
        
        time_data['hours'] = time_data['hours'].apply(lambda x: x[:-1])

        time_data['시간'] = time
        time_data['date'] = time_data['시간'] + ' ' + time_data['hours'] + ':00:00'
        time_data['hours'] = time_data['hours'].astype('int64')
        time_data['태양광명'] = region
        time_data.sort_values('hours')

        col_return = ['date', '발전량', '태양광명']

    #     print(time_data[col_return])
        data_energy = pd.concat([data_energy, time_data[col_return]])
    
    return data_energy


In [150]:
y_all = pd.DataFrame()
for i in range(10):
    x = get_energy_json(i+1, 1000)
    y = get_energy_data(x)
    y_all = pd.concat([y_all, y])
print(y_all.shape)

(74904, 3)


In [151]:
names = y_all['태양광명'].value_counts().index.to_list()
for name in names:
    print(y_all[y_all['태양광명'] == name].iloc[:2, :])
    print(y_all[y_all['태양광명'] == name].iloc[-2:, :])
    print('\n')


                  date  발전량      태양광명
0  2017-07-01 10:00:00  161  당진매립장태양광
1  2017-07-01 11:00:00  509  당진매립장태양광
                  date  발전량      태양광명
22  2020-07-26 8:00:00  205  당진매립장태양광
23  2020-07-26 9:00:00  278  당진매립장태양광


                  date  발전량        태양광명
0  2017-07-01 10:00:00  256  수원하수처리장태양광
1  2017-07-01 11:00:00  481  수원하수처리장태양광
                  date  발전량        태양광명
22  2020-06-30 8:00:00   62  수원하수처리장태양광
23  2020-06-30 9:00:00  155  수원하수처리장태양광


                  date   발전량           태양광명
0  2018-01-01 10:00:00   776  동해하수종말처리장 태양광
1  2018-01-01 11:00:00  1361  동해하수종말처리장 태양광
                  date  발전량           태양광명
22  2020-06-30 8:00:00   50  동해하수종말처리장 태양광
23  2020-06-30 9:00:00  184  동해하수종말처리장 태양광




# 여기 밑으로는 무시